# 전처리

## 1. 레이블별 폴더 만들기

In [ ]:
import pandas as pd

train_df = pd.read_csv("train_df.csv")
print(len(train_df["label"].unique()))  # type = numpy.array
label_list = train_df["label"].unique().tolist()
len(label_list)  # type = list

In [ ]:
import os

def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

for i in range(len(label_list)):  # 레이블 개수 만큼 
    createFolder(f'./train/{label_list[i]}')  # 레이블 폴더를 생성 

## 2. 이미지 파일을 레이블에 맞게 각 폴더로 옮기기

In [ ]:
train_folder = os.listdir('./train/')
train_folder = sorted(train_folder)
len(train_folder)  # -폴더개수 88 , 사진개수 = 4277개

In [ ]:
import shutil

for i in range(len(train_folder) - 88):  # 폴더 생성한것 88개 뺴주는 겁니다.
    
    if train_folder[i][-3:] == "png":   # 확장자가 png면 
        label = train_df.loc[train_df["file_name"] == f"{train_folder[i]}"]["label"][i]  # train_df에서 이미지 이름에 맞는 label을 불러와 저장
        file_source = f'./train/{train_folder[i]}'  # train 폴더에 있는 해당 이미지를
        file_destination = f'./train/{label}/'  # 해당 label 폴더로 이동 
        shutil.move(file_source, file_destination)  # 이동 실행

## 3. 오버샘플링
- good 데이터 200개 이상 but 나머지 데이터들 4~15개
- 데이터 불균형을 해소해주기 위하여 실행

In [ ]:
import pandas as pd
import os
import shutil
import time

In [ ]:
train_df = pd.read_csv("train_df.csv")
label_list = train_df["label"].unique().tolist()
label_list = sorted(label_list)
len(label_list)  # type = list

In [ ]:
# 폴더 안 파일 개수를 가져오는 함수
def get_num_of_files(directory):
    dirListing = os.listdir(directory)
    return len(dirListing)


# 15개 good 폴더의 파일 개수만 출력해보기
for label in label_list:
    num_files = get_num_of_files(f'train_state/{label}/')
    if num_files > 50:
        print(num_files)

### -> good 중의 최솟값인 200에 맞춰서 이상치를 오버샘플링해준다.

In [ ]:
# 이상치 폴더 이름 리스트
anomaly_label_list = []

for label in label_list:
    if label[-4:] != 'good':
        anomaly_label_list.append(label)


len(anomaly_label_list) # 88-15

In [ ]:
# 이상치 폴더 73개에 대해 for loop
# 폴더 내 파일들을 복사하되, 폴더 내 파일 개수가 200개 이상이 되면 stop

for dir in anomaly_label_list:
    current_dir = f'train_state/{dir}'
    
    original_file_list = []
    for file in os.listdir(current_dir):
        if len(file) == 9:
            original_file_list.append(file)

    whole_file_list = os.listdir(current_dir)

    while len(whole_file_list) < 200:
        for file in original_file_list:
            t = float(time.time())
            shutil.copyfile(f'{current_dir}/{file}', f'{current_dir}/{t}-{file}')
        whole_file_list = os.listdir(current_dir)

    print(f'{dir} complete')

# 모델링

## Library

In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np

import pandas as pd
import cv2
import os
import math
import scipy as sp
import PIL

from glob import glob
from tqdm import tqdm

from tensorflow.keras import models, layers, Model
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D
from tensorflow.keras.layers import Flatten, Dense, Dropout, ZeroPadding2D

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import EfficientNetB3, ResNet50V2, DenseNet121, VGG19

## Directory

In [ ]:
CurrentDirectory = "./"

train_directory = CurrentDirectory + 'train/'
test_directory  = CurrentDirectory + 'test/'
model_directory = CurrentDirectory + 'model/'
tensorboard_directory = CurrentDirectory + 'Tensorboard'

## Config

In [ ]:
# batch_size: 한번에 forward & Backword 하는 샘플의 수
batch_size = 32 # 메모리 문제 때문에 AWS:16~Colab32

# Epochs 수
epochs = 50

# Weight 조절 parameter
LearningRate = 1e-3 # 0.001
Decay = 1e-6

# 이미지 데이터 입력값
img_width = 384
img_height = 384

## ImageDataGenerator

In [ ]:
# 학습 도중 이미지 임의 변형 및 정규화 적용 가능
# 1. 이미지를 전부다 불러서 램 (메모리)에 올릴 수 없기 때문
# 2. 이미지는 Augmentation을 해주는게 좋아서
DATAGEN_TRAIN = ImageDataGenerator(
    rescale=1./255,
    rotation_range=70, # 오버샘플링 해주고나서 20->70으로 높게 줌
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    featurewise_center=True,
    featurewise_std_normalization=True,
    data_format="channels_last",
    validation_split=0.10) # Train / Validation

# Test 데이터에는 Augmentation 적용하지 않음
DATAGEN_TEST = ImageDataGenerator(
    rescale=1./255,
    featurewise_center=True,
    featurewise_std_normalization=True,
    data_format="channels_last")

In [ ]:
# flow_from_directory - 이미지가 폴더별로 분류되어 있을 경우 subdirectory 이름에 맞춰 자동으로 target class 만듦
# Generator의 instance 생성 (Train)
TRAIN_GENERATOR = DATAGEN_TRAIN.flow_from_directory(
    train_directory,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode= "categorical",
    subset = "training")

VALID_GENERATOR = DATAGEN_TRAIN.flow_from_directory(
    train_directory,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode="categorical",
    subset = "validation")

# Generator의 instance 생성 (Test)
TEST_GENERATOR = DATAGEN_TEST.flow_from_directory(
    test_directory,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    shuffle = False,
    class_mode='categorical')

In [ ]:
# VALID_GENERATOR가 target class 모두 포함하는지 확인
len(pd.Series( VALID_GENERATOR.labels ).value_counts())

# Models
- EfficientNetB3, ResNet50V2, DenseNet121, VGG19 총 4개의 모델 사용

#### Callback

In [ ]:
# Call-back 함수
# CheckPoint: Epoch 마다 val_loss를 확인하여, 값이 향상되었을 경우에만 저장
CP = ModelCheckpoint(filepath=model_directory+'Efnet-{epoch:03d}-{val_loss:.4f}-{val_acc:.4f}.hdf5',
            monitor='val_loss', verbose=1, save_best_only=True, mode='min')

# 학습과정 진행사항 확인
TB = TensorBoard(log_dir=tensorboard_directory, write_graph=True, write_images=True)

# 모델의 개선이 없을 경우 Learning rate 조절
LR = ReduceLROnPlateau(monitor='val_loss',factor=0.8,patience=3, verbose=1, min_lr=1e-7)

CALLBACK = [CP, TB, LR]

## EfficientNetB3

In [ ]:
Model_1 = tf.keras.applications.efficientnet.EfficientNetB3(
    include_top=None,
    weights='imagenet',
    input_tensor=None,
    input_shape=(img_width,img_height,3),
    pooling=None)

x = GlobalAveragePooling2D()(Model_1.output)
predictions = Dense(88, activation='softmax')(x)

Efnet = Model(inputs=Model_1.input, outputs=predictions)

# learning parameter를 더하여 최종 model compile
Efnet.compile(optimizer=
         SGD(learning_rate=LearningRate, decay=Decay, momentum=0.9, nesterov=True), 
         loss='categorical_crossentropy',
         metrics=['acc'])

# Training Start
Efnet.fit(
        TRAIN_GENERATOR,
        # 데이터가 너무 클 경우 1-epoch을 못하는 경우
        # steps_per_epoch=TRAIN_GENERATOR.n / batch_size,
        epochs=epochs,
        callbacks=CALLBACK,
        shuffle=True, # Training에 패턴이 존재하면 overfit이 잘 되기 때문에, Shuffle 사용해야함. 단 test에는 절대 X
        validation_data=VALID_GENERATOR)

### ResNet50V2

In [ ]:
Model_2 = tf.keras.applications.ResNet50V2(
    include_top=None,
    weights='imagenet',
    input_tensor=None,
    input_shape=(img_width,img_height,3),
    pooling=None)

x = GlobalAveragePooling2D()(Model_2.output)
predictions = Dense(88, activation='softmax')(x)

Resnet = Model(inputs=Model_2.input, outputs=predictions)

# learning parameter를 더하여 최종 model compile
Resnet.compile(optimizer=
         SGD(learning_rate=LearningRate, decay=Decay, momentum=0.9, nesterov=True), 
         loss='categorical_crossentropy',
         metrics=['acc'])

# Training Start
Resnet.fit(
        TRAIN_GENERATOR,
        # 데이터가 너무 클 경우 1-epoch을 못하는 경우
        # steps_per_epoch=TRAIN_GENERATOR.n / batch_size,
        epochs=epochs,
        callbacks=CALLBACK,
        shuffle=True, # Training에 패턴이 존재하면 overfit이 잘 되기 때문에, Shuffle 사용해야함. 단 test에는 절대 X
        validation_data=VALID_GENERATOR)

### DenseNet121

In [ ]:
Model_3 = tf.keras.applications.DenseNet121(
    include_top=None,
    weights='imagenet',
    input_tensor=None,
    input_shape=(img_width,img_height,3),
    pooling=None)

x = GlobalAveragePooling2D()(Model_3.output)
predictions = Dense(88, activation='softmax')(x)

Dsnet = Model(inputs=Model_3.input, outputs=predictions)

# learning parameter를 더하여 최종 model compile
Dsnet.compile(optimizer=
         SGD(learning_rate=LearningRate, decay=Decay, momentum=0.9, nesterov=True), 
         loss='categorical_crossentropy',
         metrics=['acc'])

# Training Start
Dsnet.fit(
        TRAIN_GENERATOR,
        # 데이터가 너무 클 경우 1-epoch을 못하는 경우
        # steps_per_epoch=TRAIN_GENERATOR.n / batch_size,
        epochs=epochs,
        callbacks=CALLBACK,
        shuffle=True, # Training에 패턴이 존재하면 overfit이 잘 되기 때문에, Shuffle 사용해야함. 단 test에는 절대 X
        validation_data=VALID_GENERATOR)

### VGG19

In [ ]:
Model_4 = tf.keras.applications.VGG19(
    include_top=None,
    weights='imagenet',
    input_tensor=None,
    input_shape=(img_width,img_height,3),
    pooling=None)

x = GlobalAveragePooling2D()(Model_4.output)
predictions = Dense(88, activation='softmax')(x)

Vgg19 = Model(inputs=Model_4.input, outputs=predictions)

# learning parameter를 더하여 최종 model compile
Vgg19.compile(optimizer=
         SGD(learning_rate=LearningRate, decay=Decay, momentum=0.9, nesterov=True), 
         loss='categorical_crossentropy',
         metrics=['acc'])

# Training Start
Vgg19.fit(
        TRAIN_GENERATOR,
        # 데이터가 너무 클 경우 1-epoch을 못하는 경우
        # steps_per_epoch=TRAIN_GENERATOR.n / batch_size,
        epochs=epochs,
        callbacks=CALLBACK,
        shuffle=True, # Training에 패턴이 존재하면 overfit이 잘 되기 때문에, Shuffle 사용해야함. 단 test에는 절대 X
        validation_data=VALID_GENERATOR)

In [ ]:
Efnet.load_weights("EfnetB3-008-0.0134-0.9951.hdf5")
Resnet.load_weights(")
Dsnet.load_weights("DenseNet121-019-0.0042-0.9984.hdf5")
Vgg19.load_weights(" model_vgg19-036-0.0489-0.9850.hdf5")

# prediction

In [ ]:
prediction_Efnet = Efnet.predict_generator(TEST_GENERATOR, verbose=1)
prediction_Resnet = Resnet.predict_generator(TEST_GENERATOR, verbose=1)
prediction_Dsnet = Dsnet.predict_generator(TEST_GENERATOR, verbose=1)
prediction_Vgg19 = Vgg19.predict_generator(TEST_GENERATOR, verbose=1)

# Submit

In [ ]:
train_y = pd.read_csv("train_df.csv")

train_labels = train_y["label"]

label_unique = sorted(np.unique(train_labels))
label_unique = {key:value for key,value in zip(label_unique, range(len(label_unique)))}

train_labels = [label_unique[k] for k in train_labels]

label_decoder = {val:key for key, val in label_unique.items()}

In [ ]:
f_pred = []

### 모델에 맞게 prediction 값 변경
for i in range(len(prediction_Efnet)):
    f_pred.append(prediction_Efnet[i].argmax())
    
f_result = [label_decoder[result] for result in f_pred]

In [ ]:
submission = pd.read_csv("sample_submission.csv")

submission["label"] = f_result

submission.to_csv("원하는이름.csv", index = False)

# Ensemble - SoftVoting
- VGG19는 성능이 너무 안좋아서 제외

In [ ]:
Prediction = (0.5 * prediction_Efnet) + (0.25 * prediction_Resnet) + (0.25 * prediction_Dsnet)

In [ ]:
train_y = pd.read_csv("train_df.csv")

train_labels = train_y["label"]

label_unique = sorted(np.unique(train_labels))
label_unique = {key:value for key,value in zip(label_unique, range(len(label_unique)))}

train_labels = [label_unique[k] for k in train_labels]

label_decoder = {val:key for key, val in label_unique.items()}

In [ ]:
f_pred = []

for i in range(len(prediction)):
    f_pred.append(prediction[i].argmax())
    
f_result = [label_decoder[result] for result in f_pred]

In [ ]:
submission = pd.read_csv("sample_submission.csv")

submission["label"] = f_result

submission.to_csv("원하는이름.csv", index = False)